Question 3 

A

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import binarize

mnist_data = np.load("MNIST_data.npy")
mnist_labels = np.load("MNIST_labels.npy")

mnist_data = mnist_data/255
mnist_data = binarize(mnist_data, threshold = 0.5)



In [2]:
data_train, data_temp, labels_train, labels_temp = train_test_split(mnist_data, mnist_labels, test_size=0.4, random_state=42)
data_dev, data_test, labels_dev, labels_test = train_test_split(data_temp, labels_temp, test_size=0.5, random_state=42)


In [3]:
def em_bernoulli_mixture(X, M, max_iter=100, tol=1e-4):
    n_samples, n_features = X.shape
    pis = np.random.rand(M, n_features)
    weights = np.full(M, 1/M)
    log_likelihood = 0

    for i in range(max_iter):
        responsibilities = np.zeros((n_samples, M))
        for j in range(M):
            likelihood = np.prod(pis[j]**X * (1-pis[j])**(1-X), axis=1)
            responsibilities[:, j] = weights[j] * likelihood

        responsibilities /= (responsibilities.sum(axis=1, keepdims=True) + 1e-10)

        effective_n = responsibilities.sum(axis=0)
        weights = effective_n / n_samples
        pis = (responsibilities.T @ X) / (effective_n[:, np.newaxis] + 1e-10)

        new_log_likelihood = np.sum(np.log(np.maximum(responsibilities.sum(axis=1), 1e-10)))
        if np.abs(new_log_likelihood - log_likelihood) < tol:
            break
        log_likelihood = new_log_likelihood

    return pis, weights, log_likelihood

In [4]:
print("Shapes of datasets:")
print("X_train:", data_train.shape)
print("X_dev:", data_dev.shape)
print("X_test:", data_test.shape)
print("y_train:", labels_train.shape)
print("y_dev:", labels_dev.shape)
print("y_test:", labels_test.shape)

Shapes of datasets:
X_train: (42000, 784)
X_dev: (14000, 784)
X_test: (14000, 784)
y_train: (42000,)
y_dev: (14000,)
y_test: (14000,)


In [13]:
def bernoulli(data, labels, M):
    unique_labels = np.unique(labels)
    n_features = data.shape[1]
    models = {c: [] for c in unique_labels}

    for c in unique_labels:
        data_c = data[labels == c]
        if M == 1:
            p = np.mean(data_c, axis = 0)
            models[c].append(p)
        else:
            for m in range(M):
                p = np.random.rand(n_features)
                models[c].append(p)
    return models

In [7]:
M_vals = [1,5,10,20]
class_models = {}

for M in M_vals:
    models = bernoulli(data_train, labels_train, M)
    class_models[M] = models

data_combined = np.concatenate((data_train, data_dev))
labels_combines = np.concatenate((labels_train, labels_dev))
optimal_M = 5
final_models = bernoulli(data_combined, labels_combines, optimal_M)

In [8]:
def log_bernoulli(x, p):
    return x * np.log(p+ 1e-9) + (1-x) * np.log(1-p + 1e-9)

In [15]:
def classify(data, models):
    predictions = []
    for i in range(data.shape[0]):
        max_prob = -np.inf
        pred_class = None
        for c in models:
            prob_c = np.log(1/10)
            for p in models[c]:
                prob_c += np.sum(log_bernoulli(data[i], p))
            if prob_c > max_prob:
                max_prob = prob_c
                pred_class = c
        predictions.append(pred_class)
    return np.array(predictions)
    

In [16]:
def error_calc(predictions, labels):
    return np.mean(predictions == labels)

In [19]:
M_vals = [1,5,10,20]
class_models = {}

for M in M_vals:
    models = bernoulli(data_train, labels_train, M)
    class_models[M] = models

data_combined = np.concatenate((data_train, data_dev))
labels_combines = np.concatenate((labels_train, labels_dev))
optimal_M = 1
final_models = bernoulli(data_combined, labels_combines, optimal_M)

predictions1= classify(data_test, final_models)
accuracy = np.mean(predictions1 == labels_test)
print("Test set accuracy:", accuracy)

Test set accuracy: 0.8364285714285714


In [21]:
M_vals = [1,5,10,20]
class_models = {}

for M in M_vals:
    models = bernoulli(data_train, labels_train, M)
    class_models[M] = models

data_combined = np.concatenate((data_train, data_dev))
labels_combines = np.concatenate((labels_train, labels_dev))
optimal_M = 5
final_models = bernoulli(data_combined, labels_combines, optimal_M)

predictions5= classify(data_test, final_models)
accuracy = np.mean(predictions5 == labels_test)
print("Test set accuracy:", accuracy)

Test set accuracy: 0.10721428571428572


In [22]:
M_vals = [1,5,10,20]
class_models = {}

for M in M_vals:
    models = bernoulli(data_train, labels_train, M)
    class_models[M] = models

data_combined = np.concatenate((data_train, data_dev))
labels_combines = np.concatenate((labels_train, labels_dev))
optimal_M = 10
final_models = bernoulli(data_combined, labels_combines, optimal_M)

predictions10= classify(data_test, final_models)
accuracy = np.mean(predictions10 == labels_test)
print("Test set accuracy:", accuracy)

Test set accuracy: 0.09342857142857143


In [23]:
M_vals = [1,5,10,20]
class_models = {}

for M in M_vals:
    models = bernoulli(data_train, labels_train, M)
    class_models[M] = models

data_combined = np.concatenate((data_train, data_dev))
labels_combines = np.concatenate((labels_train, labels_dev))
optimal_M = 20
final_models = bernoulli(data_combined, labels_combines, optimal_M)

predictions20 = classify(data_test, final_models)
accuracy = np.mean(predictions20 == labels_test)
print("Test set accuracy:", accuracy)

Test set accuracy: 0.11078571428571428


The optimal M is M = 10 as it has the lowest test set error rate.

In [11]:
M_vals = [5,10,20]

results = {}
for m in M_vals:
    models = {}
    for label in np.unique(labels_train):
        data_class = data_train[labels_train == label]
        pis, weights, _ = em_bernoulli_mixture(data_class, m)
        models[label] = pis
        
    predictions = classify(data_test, models)
    error = error_calc(predictions, labels_test)
    results[m] = error

for M, err in results.items():
    print(f'Test accuracy for M = {M}: {err}')
    

Test accuracy for M = 5: 0.9007857142857143
Test accuracy for M = 10: 0.9007857142857143
Test accuracy for M = 20: 0.9007857142857143


B

In [26]:
from sklearn.linear_model import LogisticRegression

c_vals = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
best_error_rate = float('inf')
best_c = None

for c in c_vals:
    log_reg = LogisticRegression(fit_intercept=True, C = c, penalty = 'l2', multi_class = 'multinomial', solver = 'lbfgs', max_iter = 1000)
    log_reg.fit(data_train, labels_train)

    labels_predicted_dev = log_reg.predict(data_dev)
    correct_predictions_dev = np.sum(labels_predicted_dev == labels_dev)
    error_rate_dev = 1- (correct_predictions_dev / labels_dev.size)

    if error_rate_dev < best_error_rate:
        best_error_rate = error_rate_dev
        best_c = c

In [27]:
log_reg_optimal = LogisticRegression(fit_intercept = True, C = best_c, penalty = 'l2', multi_class = 'multinomial', solver = 'lbfgs', max_iter = 1000)
log_reg_optimal.fit(data_combined, labels_combines)

labels_pred_test = log_reg_optimal.predict(data_test)
correct_pred_test = np.sum(labels_pred_test == labels_test)
error_rate_test = 1 - (correct_pred_test / labels_test.size)
error_rate_test


0.08221428571428568

The logistic regression had a slightly higher error rate than the best M using bernoulli but was faster in run time.

In [28]:
print(best_c)

0.1


In [29]:
data_center = mnist_data - np.mean(mnist_data)

covariance_mtx = np.cov(data_center.T)
eigenvalues, eigenvectors = np.linalg.eigh(covariance_mtx)

sorted_i = np.argsort(eigenvalues)[::-1]
sorted_vals = eigenvalues[sorted_i]
sorted_vecs = eigenvectors[:, sorted_i]


In [30]:
def project(data, pcs):
    return np.dot(data, pcs)

k_vals = [10, 20, 50, 100, 200, 300]
c_vals = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

best_error_rate = float('inf')
best_k = None
best_c = None

for k in k_vals:
    principal_components = sorted_vecs[:, :k]
    data_train_pca = project(data_train - np.mean(mnist_data, axis=0), principal_components)
    data_dev_pca = project(data_dev - np.mean(mnist_data, axis = 0), principal_components)

    for c in c_vals:
        log_reg_pca = LogisticRegression(fit_intercept = True, C=c, penalty = 'l2', multi_class = 'multinomial', solver = 'lbfgs', max_iter = 1000)
        log_reg_pca.fit(data_train_pca, labels_train)

        labels_pred_dev_pca = log_reg_pca.predict(data_dev_pca)
        correct_predictions_dev = np.sum(labels_pred_dev_pca == labels_dev)
        error_rate_dev = 1 - (correct_predictions_dev /labels_dev.size)

        if error_rate_dev < best_error_rate:
            best_error_rate = error_rate_dev
            best_k = k
            best_c = c


data_combined_pca = np.dot(data_combined, sorted_vecs[:, :best_k])
log_reg_final = LogisticRegression(fit_intercept = True, C = best_c, penalty = 'l2', multi_class = 'multinomial', solver = 'lbfgs', max_iter = 1000)
log_reg_final.fit(data_combined_pca, labels_combines)

data_test_pca = np.dot(data_test - np.mean(mnist_data, axis = 0), sorted_vecs[:, :best_k])
labels_pred_test_pca = log_reg_final.predict(data_test_pca)
correct_predictions_test = np.sum(labels_pred_test_pca == labels_test)
error_rate_test_pca = 1 - (correct_predictions_test / labels_test.size)

    

In [31]:
best_k

200

In [32]:
best_c

0.1

In [33]:
error_rate_test_pca

0.23985714285714288

The logistic regression using pca had a higher error rate than the previous logistic regression. This could be because with PCA some values are omitted as we use the most relevant eigenvalues. However with pca the run time was faster.

In [34]:

from scipy.special import softmax, logsumexp

def sgd_logistic_regression(X_train, y_train, X_val, y_val, batch_size, learning_rate, lambda_reg, epochs):
    theta = np.zeros((X_train.shape[1],))
    
    def negative_log_likelihood(X, y, theta):
        logits = X.dot(theta)
        log_likelihoods = -logsumexp(np.c_[logits, np.zeros_like(logits)], axis=1) + logits * y
        return -np.sum(log_likelihoods) / len(y) + (lambda_reg / 2) * np.sum(theta ** 2)

    def gradient(X, y, theta):
        logits = X.dot(theta)
        predictions = softmax(logits[:, np.newaxis], axis=1)[:, 0]
        gradient = -X.T.dot(y - predictions) / len(y) + lambda_reg * theta
        return gradient

    for epoch in range(epochs):
        permutation = np.random.permutation(len(X_train))
        X_train_shuffled = X_train[permutation]
        y_train_shuffled = y_train[permutation]
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train_shuffled[i:i + batch_size]
            y_batch = y_train_shuffled[i:i + batch_size]
            theta -= learning_rate * gradient(X_batch, y_batch, theta)

        train_nll = negative_log_likelihood(X_train, y_train, theta)
        val_nll = negative_log_likelihood(X_val, y_val, theta)
        print(f'Epoch {epoch+1}, Training NLL: {train_nll}, Validation NLL: {val_nll}')
        
        if epoch > 0 and val_nll >= previous_val_nll:
            break
        previous_val_nll = val_nll
    
    return theta

batch_size = 128
learning_rate = 0.01
lambda_reg = 0.1
epochs = 100


theta = sgd_logistic_regression(data_train, labels_train, data_dev, labels_dev, batch_size, learning_rate, lambda_reg, epochs)

def predict(X, theta):
    theta = np.atleast_2d(theta).T

    
    logits = X.dot(theta)
    probs = softmax(logits, axis = 1)

    if probs.shape[1] == 1:
        return (probs>0.5).astype(int).flatten()

    return np.argmax(probs, axis = 1)


labels_pred_test = predict(data_test, theta)
error_rate_test = np.mean(labels_pred_test != labels_test)




Epoch 1, Training NLL: -1128.0685545274123, Validation NLL: -1135.3504190761173
Epoch 2, Training NLL: -1712.7912016736295, Validation NLL: -1725.3096590091018
Epoch 3, Training NLL: -2015.533791276489, Validation NLL: -2031.8201906286245
Epoch 4, Training NLL: -2172.5017538899083, Validation NLL: -2191.5103033908235
Epoch 5, Training NLL: -2253.578576151778, Validation NLL: -2274.5345441837662
Epoch 6, Training NLL: -2295.0482556751504, Validation NLL: -2317.385731852646
Epoch 7, Training NLL: -2316.913535395464, Validation NLL: -2340.2652474741967
Epoch 8, Training NLL: -2328.072869686831, Validation NLL: -2352.142326435105
Epoch 9, Training NLL: -2333.8116784441877, Validation NLL: -2358.3960516519187
Epoch 10, Training NLL: -2336.8332414604906, Validation NLL: -2361.7786182743544
Epoch 11, Training NLL: -2338.490124390355, Validation NLL: -2363.709265088958
Epoch 12, Training NLL: -2339.2659717390693, Validation NLL: -2364.6735547232706
Epoch 13, Training NLL: -2339.70636319715, Va

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import time

def train_and_evaluate_sgd(X_train, y_train, X_val, y_val, X_test, y_test, lambda_vals, learning_rate, epochs, batch_size):
    best_lambda = None
    best_validation_error = float("inf")
    training_errors = []
    validation_errors = []
    test_error = None

    for lambda_reg in lambda_vals:
        print(f"Training with λ = {lambda_reg}")
        start_time = time.time()
        theta = sgd_logistic_regression(X_train, y_train, X_val, y_val, batch_size, learning_rate, lambda_reg, epochs)
        training_time = time.time() - start_time
        
        labels_pred_val = predict(X_val, theta)
        validation_error = np.mean(labels_pred_val != y_val)

        if validation_error < best_validation_error:
            best_validation_error = validation_error
            best_lambda = lambda_reg
            test_error = np.mean(predict(X_test, theta) != y_test)
        
    return best_lambda, test_error

lambda_vals = [0, 10, 100]

learning_rate = 0.01
epochs = 100
batch_size = 64

best_lambda, test_error = train_and_evaluate_sgd(data_train, labels_train, data_dev, labels_dev, data_test, labels_test, lambda_vals, learning_rate, epochs, batch_size)

print(f"Test error for best λ ({best_lambda}): {test_error:.4f}")


Training with λ = 0
Epoch 1, Training NLL: -3075.331249959077, Validation NLL: -3092.3688277232136
Epoch 2, Training NLL: -6151.149952161459, Validation NLL: -6185.232746015626
Epoch 3, Training NLL: -9226.588910669641, Validation NLL: -9277.713071484373
Epoch 4, Training NLL: -12300.58506292783, Validation NLL: -12368.741957511162
Epoch 5, Training NLL: -15375.318328333333, Validation NLL: -15460.509217968747
Epoch 6, Training NLL: -18452.295416249995, Validation NLL: -18554.535917477675
Epoch 7, Training NLL: -21527.426509289435, Validation NLL: -21646.70589770089
Epoch 8, Training NLL: -24602.804683333325, Validation NLL: -24739.122412700886
Epoch 9, Training NLL: -27676.90410908481, Validation NLL: -27830.25147388392
Epoch 10, Training NLL: -30751.163972135408, Validation NLL: -30921.546928459808
Epoch 11, Training NLL: -33826.61653135787, Validation NLL: -34014.03779407364
Epoch 12, Training NLL: -36900.79816720981, Validation NLL: -37105.24934845981
Epoch 13, Training NLL: -39976

In [36]:
error_rate_test

0.8864285714285715

The best error rate achieved seems to be with the Bernoulli mixtures. The Stochastic gradient descent was faster than the logistic regression but had a higher error rate.